In [1]:
from dotenv import load_dotenv
import os
load_dotenv()



True

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = os.getenv('WEAVIATE_URL')
weaviate_api_key = os.getenv('weaviate_api_key')

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()


embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding  = HuggingFaceEmbeddings(model_name = embedding_model_id)

c:\Users\sjitb\OneDrive\Documents\KaggleData\Projects\GenAI\RAG_application_using_Mistral_AI_and_Weviatedb\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader("state_of_union.txt", encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
vector_db = WeaviateVectorStore.from_documents(documents=docs, embedding=embedding, client=client,by_text=False)
retriever = vector_db.as_retriever()

In [6]:
vector_db.similarity_search('what is content?', k=3)

[Document(metadata={'source': 'state_of_union.txt'}, page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully. \n\nWe spent months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. \n\nI spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  \n\nWe countered Russia’s lies with truth.   \n\nAnd now that he has acted the free world is holding him accountable. \n\nAlong with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Z

In [7]:
from langchain.prompts import ChatPromptTemplate

template = 'You are an assistant for question-answering task.Use the following pieces of retrieved contect to answer the question.' \
            'If you dont know the answer just say that you dont know .Use ten sentences maximum and keep the answer concise' \
            'Question : {question}'\
            'Context : {context}'\
            'Answer ::'

prompt = ChatPromptTemplate.from_template(template=template)
print(prompt)


input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are an assistant for question-answering task.Use the following pieces of retrieved contect to answer the question.If you dont know the answer just say that you dont know .Use ten sentences maximum and keep the answer conciseQuestion : {question}Context : {context}Answer ::'), additional_kwargs={})]


In [8]:
from langchain_community.llms.huggingface_hub import HuggingFaceHub
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model='gemini-1.5-pro')

c:\Users\sjitb\OneDrive\Documents\KaggleData\Projects\GenAI\RAG_application_using_Mistral_AI_and_Weviatedb\venv\Lib\site-packages\langchain_google_genai\chat_models.py:1022: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = ({'context' : retriever, 'question' : RunnablePassthrough()} |
             prompt | model | StrOutputParser())

In [10]:
rag_chain.invoke('What is the documnet about ?')

"This document is the State of the Union address.  The speaker discusses the Cancer Moonshot initiative, aiming to cut the cancer death rate by 50% over 25 years.  Capping the cost of insulin at $35 a month for those with Type 1 diabetes is another key point.  The address also mentions ARPA-H, a new research agency modeled after DARPA, to drive breakthroughs in diseases like cancer and Alzheimer's.  The speaker expresses optimism about America's future and its ability to overcome challenges.  The overall tone is one of strength, resilience, and unity.  The setting is the Capitol, described as the citadel of democracy. The speech emphasizes American exceptionalism and the nation's capacity to turn crises into opportunities."